In [4]:
import dgl
import numpy as np
import os

In [2]:
prefix = "/localtmp/as9rw/datasets/botnet_temp"

In [3]:
adv_graphs, _ = dgl.load_graphs(os.path.join(prefix, "dgl_adv.hdf5"))
victim_graphs, _ = dgl.load_graphs(os.path.join(prefix, "dgl_victim.hdf5"))

In [50]:
all_graphs = []
for g in adv_graphs:
    all_graphs.append(g)
for g in victim_graphs:
    all_graphs.append(g)

In [51]:
len(all_graphs)

488

In [8]:
def get_em(filename):
    names, values = [], []
    with open(filename, 'r') as f:
        for line in f:
            n, v = line.rstrip("\n").split(',')
            values.append(float(v))
            names.append(n)
        
        names = np.array(names)
        values = np.array(values)
    return names, values

In [12]:
adv_f = get_em("./adv_info.txt")
victim_f = get_em("./victim_info.txt")

In [14]:
# Combine all data together
combined_names = np.concatenate((adv_f[0], victim_f[0]))

In [16]:
combined_vals = np.concatenate((adv_f[1], victim_f[1]))

In [40]:
below_val = 0.0061
above_val = 0.0074

print(np.sum(combined_vals < below_val))
print(np.sum(combined_vals > above_val))

# Calculate wastage
print(np.sum(np.logical_and(combined_vals >= below_val, combined_vals <= above_val)))

188
190
110


In [41]:
zero_ids = np.where(combined_vals < below_val)[0]
one_ids = np.where(combined_vals > above_val)[0]

# Shuffle things up first
szero = np.random.permutation(zero_ids)
sone = np.random.permutation(one_ids)

print(len(szero), len(sone))

# Make 50:50 victim/adv splits
sp_z = int(0.5 * len(szero))
sp_o = int(0.5 * len(sone))

victim_z = szero[:sp_z]
adv_z = szero[sp_z:]

victim_o = sone[:sp_o]
adv_o = sone[sp_o:]

188 190


In [57]:
print(len(victim_z), len(victim_o))

94 95


In [58]:
print(len(adv_z), len(adv_o))

94 95


In [44]:
# Coalesce into victim/adv splits
victim = np.concatenate((victim_o, victim_z))
adv = np.concatenate((adv_o, adv_z))

In [53]:
with open("./victim_info_new.txt", 'w') as f:
    for ii in victim:
        f.write(combined_names[ii] + "," + str(combined_vals[ii]) + "\n")

In [54]:
with open("./adv_info_new.txt", 'w') as f:
    for ii in adv:
        f.write(combined_names[ii] + "," + str(combined_vals[ii]) + "\n")

In [55]:
victim_gs = [all_graphs[ii] for ii in victim]
dgl.save_graphs(os.path.join(prefix, "dgl_victim_new.hdf5"), victim_gs)

In [56]:
adv_gs = [all_graphs[ii] for ii in adv]
dgl.save_graphs(os.path.join(prefix, "dgl_adv_new.hdf5"), adv_gs)

In [59]:
len(victim_gs)

189

In [60]:
len(adv_gs)

189